In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,8164.80,-0.01,-0.01,-0.02,-0.01,-0.00,-0.00,0.00
3,-0.01,1626.35,-0.00,0.01,0.00,0.00,0.00,-0.00
4,-0.01,-0.00,312.32,-0.01,-0.00,0.00,-0.00,0.00
5,-0.02,0.01,-0.01,59.96,0.00,-0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,11.68,-0.00,0.00,0.00
7,-0.00,0.00,0.00,-0.00,-0.00,2.32,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.52,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00000,-0.00001,-0.00004,-0.00004,-0.00001,-0.00004,0.00001
3,-0.00000,1.00000,-0.00000,0.00002,0.00001,0.00003,0.00000,-0.00003
4,-0.00001,-0.00000,1.00000,-0.00004,-0.00001,0.00006,-0.00003,0.00003
5,-0.00004,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00001,0.00003,0.00006,-0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00004,0.00000,-0.00003,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04952008392636734

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.34028598e+06 3.23967186e+05 1.16184960e+04 4.11241779e+02
 1.46596650e+01 5.16993916e-01 2.04183846e-02 3.24107847e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999547,-0.029587,-0.005434,-0.001044,-0.000205,-0.000041,-0.000009,-0.000002
1,0.029382,0.998992,-0.033351,-0.006131,-0.001193,-0.000236,-0.000053,-0.000015
2,0.006360,0.032896,0.998608,-0.040016,-0.007465,-0.001473,-0.000332,-0.000089
3,0.001463,0.007331,0.039341,0.997859,-0.050767,-0.009618,-0.002147,-0.000583
4,0.000356,0.001776,0.009267,0.049607,0.996228,-0.068923,-0.014645,-0.003947
5,0.000092,0.000461,0.002400,0.012542,0.066052,0.990490,-0.116379,-0.029307
6,0.000029,0.000143,0.000748,0.003889,0.019983,0.103346,0.958265,-0.265767
7,0.000017,0.000083,0.000433,0.002258,0.011570,0.058341,0.260699,0.963584


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004531247591536003,
 0.0010078346518425096,
 0.0013919104536221916,
 0.002141482657390781,
 0.0037723878156809,
 0.009509820849826323,
 0.04173455774567514,
 0.036416472984837966]